In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
import seaborn as sn
import teotil2 as teo
import useful_rid_code as rid

sn.set_context("notebook")

# RID

## Estimating loads in unmonitored regions (parameterised)

This notebook is "parameterised" for use with Papermill. The cell below has the tag `parameters`, which means the entire notebook can be called from `01_recalculate_ospar_1990-2016_main.ipynb`.

The [TEOTIL2 model](https://nivanorge.github.io/teotil2/) is used to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere.

In [2]:
# This cell is tagged 'parameters' for use with Papermill
# https://papermill.readthedocs.io/en/latest/index.html
year = 1990
user = ""
pw = ""

In [3]:
# Parameters
user = "jes"
pw = "BeakabusNov21.."
year = 2012


## 1. Generate model input file

In [4]:
# Parameters of interest
par_list = ["Tot-N", "Tot-P"]

# Path to TEOTIL2 "core" input data
teo_fold = r"../../../teotil2/data/core_input_data"

# Ouput path for model file
ann_input_csv = f"../../../teotil2/data/norway_annual_input_data/input_data_{year}.csv"

In [5]:
engine = rid.connect_to_nivabase(user=user, pw=pw)
df = teo.io.make_input_file(
    year, engine, teo_fold, ann_input_csv, mode="nutrients", par_list=par_list
)

Connection successful.


/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## 2. Run model

In [6]:
%%time

# Run model
g = teo.model.run_model(ann_input_csv)

/opt/conda/lib/python3.9/site-packages/teotil2/model.py:67: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  df[f"trans_{par}"].between(0, 1, inclusive=True).all()


CPU times: user 6.8 s, sys: 30.1 ms, total: 6.83 s
Wall time: 6.9 s


## 3. Save results

In [7]:
# Save results as csv
res_csv = f"../../../teotil2/data/norway_annual_output_data/teotil2_results_{year}.csv"
df = teo.model.model_to_dataframe(g, out_path=res_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,...,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes,local_vol_lake_m3
0,001.1A2B,001.1A2A,2.753530,0.148800,0.033365,0.002762,0.340207,0.032497,14.555834,0.513959,...,0.0,0.0,371.4528,0.306842,0.029736,1.00,1.00,0.02205,0.00315,1.788754e+08
1,001.1A4D,001.1A4C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.613841,0.010170,...,0.0,0.0,433.3616,0.000000,0.000000,0.81,0.26,0.00000,0.00000,8.019497e+06
2,001.1M,001.1L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.806950,0.035141,...,0.0,0.0,433.3616,0.000000,0.000000,0.86,0.31,0.00000,0.00000,3.866908e+07
3,001.221Z,001.2210,0.111092,0.006003,0.001346,0.000111,0.013726,0.001311,0.450718,0.010877,...,0.0,0.0,371.4528,0.012380,0.001200,1.00,1.00,0.00000,0.00000,0.000000e+00
4,001.222Z,001.2220,0.915450,0.049471,0.011093,0.000918,0.113107,0.010804,2.295740,0.075445,...,0.0,0.0,371.4528,0.102014,0.009886,1.00,1.00,0.00000,0.00000,0.000000e+00


In [8]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query("regine in @main_list")
df2.sort_values("regine", inplace=True)

# Save
main_csv = f"../../../Results/Unmon_loads/teotil2_results_{year}_main_catchs.csv"
df2.to_csv(main_csv, index=False, encoding="utf-8")

/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [9]:
# Read station data
in_xlsx = r"../../../Data/RID_Sites_List_2017-2020.xlsx"
stn_df = pd.read_excel(in_xlsx, sheet_name="RID_All")
stn_df = stn_df.query("station_id != 38005")  # Ignore TROEMÅL2

# Get just cols of interest and drop duplicates
# (some sites are in the same regine)
stn_df = stn_df[["ospar_region", "nve_vassdrag_nr"]].drop_duplicates()

# Get catch IDs with obs data
obs_nds = set(stn_df["nve_vassdrag_nr"].values)

# Build network from input file
g, nd_list = teo.calib.build_calib_network(ann_input_csv, obs_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df["nve_vassdrag_nr"].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [10]:
# Read model output
teo_df = pd.read_csv(res_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(
    stn_df, teo_df, how="left", left_on="nve_vassdrag_nr", right_on="regine"
)

# Groupby OSPAR region
mon_df = mon_df.groupby("ospar_region").sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split("_")[0] == "accum"]
mon_df = mon_df[cols]

mon_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,147.620773,4.947769,2.540371,0.213926,81.642315,5.511816,3416.579350,51.205043,147.620773,4.947769,...,3187.316262,40.745458,725.124444,29.585803,0.870620,49.516141,4.427269,63555.61,0.000000,0.000000
NORTH SEA,2907.180912,64.595210,34.943085,2.173150,375.797490,55.376003,12152.062100,193.459793,2913.153482,65.138686,...,8863.111128,72.945104,1278.843045,197.011753,39.556744,136.745052,10.331189,23353.19,5.972570,0.543476
NORWEGIAN SEA2,2937.971693,75.568963,40.709881,3.053550,529.965302,48.617001,12350.229879,247.806215,2954.438670,77.698114,...,8865.825907,121.491100,1659.412543,272.628007,16.477551,177.225318,17.151996,45896.63,16.466978,2.129151
SKAGERAK,11230.531042,209.137397,102.034032,5.504629,3473.840556,109.487864,28834.828521,463.187563,11392.466281,226.615159,...,13968.521684,127.084539,1762.420487,2371.234585,18.828554,837.229927,53.633420,93945.27,161.935239,17.477763


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (files named `regine_{year}.csv`) don't exactly match the relevant OSPAR definitions for this analysis. This is because the "OSPAR boundaries" in the model include catchments draining to Sweden (as part of TEOTIL2 Metals - see [here](https://nivanorge.github.io/teotil2/pages/07_1000_lakes.html)), so instead of using them directly we need to aggregate based on vassdragsnummers.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [11]:
# Define "new" OSPAR regions (ranges are inclusive)
os_dict = {
    "SKAGERAK": (1, 23),
    "NORTH SEA": (24, 90),
    "NORWEGIAN SEA2": (91, 170),
    "LOFOTEN-BARENTS SEA": (171, 247),
}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]

    regs = ["%03d." % i for i in range(min_id, max_id + 1)]

    # Get data for this region
    df2 = teo_df[teo_df["regine"].isin(regs)]

    # Get just accum cols
    cols = [i for i in df2.columns if i.split("_")[0] == "accum"]
    df2 = df2[cols]

    # Add region
    df2["ospar_region"] = reg

    # Add to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby("ospar_region").sum()

os_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,618.060458,25.453705,10.569251,0.866142,13785.304369,2319.883074,21556.430086,2456.505433,628.032768,26.857546,...,7143.092949,109.764814,1782.144142,932.513831,121.196757,315.271216,38.845381,138090.89,9.972310,1.403841
NORTH SEA,6951.609750,177.154195,80.515432,5.832951,23197.161852,3780.098015,48756.653324,4135.472581,6991.427232,182.222864,...,18568.064240,173.151702,2781.265118,3170.047195,319.327818,735.961774,84.202958,59314.38,39.817482,5.068670
NORWEGIAN SEA2,7952.861622,242.009189,101.652087,8.012104,27060.950716,4465.944640,51556.963995,4951.653126,7985.020412,246.249632,...,16510.992866,239.458854,3290.412826,2343.725445,268.304403,683.486303,82.316629,113934.05,32.158791,4.240442
SKAGERAK,12890.033997,266.694916,112.948837,6.321836,9199.174461,287.589805,37464.101577,726.221670,13163.768466,299.426882,...,15101.158650,139.204984,1869.153911,6937.664824,101.498992,1021.058820,72.294058,102574.69,273.734469,32.731966


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [12]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_raw_unmonitored_loads_{year}.csv"
unmon_df.to_csv(out_csv, encoding="utf-8", index_label="ospar_region")

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_agri_diff_tot-n_tonnes,470,4044,5015,1660
accum_agri_diff_tot-p_tonnes,21,113,166,58
accum_agri_pt_tot-n_tonnes,8,46,61,11
accum_agri_pt_tot-p_tonnes,1,4,5,1
accum_all_point_tot-n_tonnes,13704,22821,26531,5725
accum_all_point_tot-p_tonnes,2314,3725,4417,178
accum_all_sources_tot-n_tonnes,18140,36605,39207,8629
accum_all_sources_tot-p_tonnes,2405,3942,4704,263
accum_anth_diff_tot-n_tonnes,480,4078,5031,1771
accum_anth_diff_tot-p_tonnes,22,117,169,73


#### 4.1.5. Aggregate values to required quantities

In [13]:
# Aggregate to match report
unmon_df["flow"] = unmon_df["accum_q_m3/s"] * 60 * 60 * 24 / 1000.0  # 1000s m3/day

unmon_df["sew_n"] = (
    unmon_df["accum_ren_tot-n_tonnes"] + unmon_df["accum_spr_tot-n_tonnes"]
)
unmon_df["sew_p"] = (
    unmon_df["accum_ren_tot-p_tonnes"] + unmon_df["accum_spr_tot-p_tonnes"]
)

unmon_df["ind_n"] = unmon_df["accum_ind_tot-n_tonnes"]
unmon_df["ind_p"] = unmon_df["accum_ind_tot-p_tonnes"]

unmon_df["fish_n"] = unmon_df["accum_aqu_tot-n_tonnes"]
unmon_df["fish_p"] = unmon_df["accum_aqu_tot-p_tonnes"]

unmon_df["diff_n"] = (
    unmon_df["accum_anth_diff_tot-n_tonnes"] + unmon_df["accum_nat_diff_tot-n_tonnes"]
)
unmon_df["diff_p"] = (
    unmon_df["accum_anth_diff_tot-p_tonnes"] + unmon_df["accum_nat_diff_tot-p_tonnes"]
)

new_df = unmon_df[
    ["flow", "sew_n", "sew_p", "ind_n", "ind_p", "fish_n", "fish_p", "diff_n", "diff_p"]
]

# Total for Norway
new_df.loc["NORWAY"] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(
    ["NORWAY", "LOFOTEN-BARENTS SEA", "NORTH SEA", "NORWEGIAN SEA2", "SKAGERAK"]
)

new_df.round().astype(int)

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,371276,12069,927,2288,316,54300,9382,33799,680
LOFOTEN-BARENTS SEA,91327,1169,155,104,10,12423,2149,4436,91
NORTH SEA,129809,3572,354,442,137,18761,3231,13783,217
NORWEGIAN SEA2,140918,2577,317,801,97,23091,3998,12676,287
SKAGERAK,9222,4750,101,940,72,24,4,2904,85


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [14]:
# Dict of conversion factors
con_dict = {
    ("sew", "po4"): ("p", 0.6),
    ("ind", "po4"): ("p", 0.6),
    ("fish", "po4"): ("p", 0.69),
    ("diff", "po4"): ("p", 0.246),
    ("sew", "no3"): ("n", 0.05),
    ("ind", "no3"): ("n", 0.05),
    ("fish", "no3"): ("n", 0.11),
    ("diff", "no3"): ("n", 0.625),
    ("sew", "nh4"): ("n", 0.75),
    ("ind", "nh4"): ("n", 0.75),
    ("fish", "nh4"): ("n", 0.8),
    ("diff", "nh4"): ("n", 0.055),
}

# Apply factors
for src in ["sew", "ind", "fish", "diff"]:
    for spc in ["po4", "no3", "nh4"]:
        el, fac = con_dict[(src, spc)]
        new_df[src + "_" + spc] = fac * new_df[src + "_" + el]

new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,371276,91327,129809,140918,9222
sew_n,12069,1169,3572,2577,4750
sew_p,927,155,354,317,101
ind_n,2288,104,442,801,940
ind_p,316,10,137,97,72
fish_n,54300,12423,18761,23091,24
fish_p,9382,2149,3231,3998,4
diff_n,33799,4436,13783,12676,2904
diff_p,680,91,217,287,85
sew_po4,556,93,212,190,61


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [15]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
    "  a.type, "
    "  b.name, "
    "  b.unit, "
    "  SUM(c.value * d.factor) as value "
    "FROM RESA2.RID_PUNKTKILDER a, "
    "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
    "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
    "RESA2.RID_PUNKTKILDER_INP_OUTP d "
    "WHERE a.anlegg_nr = c.anlegg_nr "
    "AND (1, a.regine) NOT IN %s "
    "AND d.in_pid = c.inp_par_id "
    "AND d.out_pid = b.out_pid "
    "AND c.year = %s "
    "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
    "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year)
)

df = pd.read_sql(sql, engine)

# Tidy
df["par"] = df["type"] + "_" + df["name"] + "_" + df["unit"]
del df["name"], df["unit"], df["type"]

# Pivot
df = df.pivot(index="vassdrag", columns="par", values="value")
df.reset_index(inplace=True)

Connection successful.


In [16]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999


# Convert vassdrag to numbers
df["vass"] = df["vassdrag"].apply(f)

# Get just the main catchments
df = df.query("vass != -999")

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PAH_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,100.49363,1.61732,NaN,1
1,002,0.013694,0.004325,NaN,3.214823,0.004211,NaN,NaN,0.511292,0.000300,...,0.000240,0.205155,0.002508,0.000115,0.012950,0.03265,475.17828,8.27060,0.733085,2
2,003,NaN,0.000041,NaN,NaN,0.000003,NaN,NaN,NaN,0.000263,...,NaN,NaN,NaN,NaN,NaN,NaN,262.21600,2.37250,NaN,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000033,0.007335,NaN,NaN,0.000083,NaN,151.79450,1.01260,0.180143,4
4,005,0.000011,0.000001,NaN,0.000014,0.000000,NaN,NaN,0.000472,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,38.84622,0.39317,NaN,5


In [17]:
def f2(x):
    if x in range(1, 24):
        return "SKAGERAK"
    elif x in range(24, 91):
        return "NORTH SEA"
    elif x in range(91, 171):
        return "NORWEGIAN SEA2"
    elif x in range(171, 248):
        return "LOFOTEN-BARENTS SEA"
    else:
        return np.nan


# Assign main catchments to OSPAR regions
df["osp_reg"] = df["vass"].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PAH_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,100.49363,1.61732,NaN,1,SKAGERAK
1,002,0.013694,0.004325,NaN,3.214823,0.004211,NaN,NaN,0.511292,0.000300,...,0.205155,0.002508,0.000115,0.012950,0.03265,475.17828,8.27060,0.733085,2,SKAGERAK
2,003,NaN,0.000041,NaN,NaN,0.000003,NaN,NaN,NaN,0.000263,...,NaN,NaN,NaN,NaN,NaN,262.21600,2.37250,NaN,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007335,NaN,NaN,0.000083,NaN,151.79450,1.01260,0.180143,4,SKAGERAK
4,005,0.000011,0.000001,NaN,0.000014,0.000000,NaN,NaN,0.000472,NaN,...,NaN,NaN,NaN,NaN,NaN,38.84622,0.39317,NaN,5,SKAGERAK


In [18]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby("osp_reg").sum()
if 0 in df.index:
    df.drop(0, inplace=True)

# Total for Norway
df.loc["NORWAY"] = df.sum(axis=0)

# Join to model results
df = new_df.join(df)

# Get cols of interest
umod_cols = ["S.P.M.", "TOC", "As", "Pb", "Cd", "Cu", "Zn", "Ni", "Cr", "Hg"]
umod_cols = [
    "%s_%s_tonn" % (i, j) for i in ["INDUSTRI", "RENSEANLEGG"] for j in umod_cols
]
cols = list(new_df.columns) + umod_cols
cols.remove("RENSEANLEGG_TOC_tonn")
cols = [i for i in cols if i in df.columns]
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,371276,91327,129809,140918,9222
sew_n,12069,1169,3572,2577,4750
sew_p,927,155,354,317,101
ind_n,2288,104,442,801,940
ind_p,316,10,137,97,72
fish_n,54300,12423,18761,23091,24
fish_p,9382,2149,3231,3998,4
diff_n,33799,4436,13783,12676,2904
diff_p,680,91,217,287,85
sew_po4,556,93,212,190,61


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [19]:
engine = rid.connect_to_nivabase(user=user, pw=pw)

# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = (
    "SELECT SUBSTR(regine, 1, 3) AS vassdrag, "
    "  SUM(value) AS value FROM ( "
    "    SELECT b.regine, "
    "           c.name, "
    "           (a.value*d.factor) AS value "
    "    FROM resa2.rid_kilder_aqkult_values a, "
    "    resa2.rid_kilder_aquakultur b, "
    "    resa2.rid_punktkilder_outpar_def c, "
    "    resa2.rid_punktkilder_inp_outp d "
    "    WHERE a.anlegg_nr = b.nr "
    "    AND (1, b.regine) NOT IN %s "
    "    AND a.inp_par_id = d.in_pid "
    "    AND c.out_pid = d.out_pid "
    "    AND name = 'Cu' "
    "    AND ar = %s) "
    "GROUP BY SUBSTR(regine, 1, 3)" % (tuple(nd_list_hack), year)
)

aq_df = pd.read_sql(sql, engine)

if len(aq_df) > 0:
    # Get vassdrag
    aq_df["vass"] = aq_df["vassdrag"].apply(f)
    aq_df = aq_df.query("vass != -999")

    # Calc OSPAR region and group
    aq_df["osp_reg"] = aq_df["vass"].apply(f2)
    aq_df.fillna(0, inplace=True)
    aq_df = aq_df.groupby("osp_reg").sum()
    del aq_df["vass"]

    # Total for Norway
    aq_df.loc["NORWAY"] = aq_df.sum(axis=0)

    # Rename
    aq_df.columns = [
        "AQUAKULTUR_Cu_tonn",
    ]

    # Join model results
    df = df.join(aq_df)

    df.round(0).astype(int).T

Connection successful.


In [20]:
# Write output
out_csv = f"../../../Results/Unmon_loads/teotil2_ospar_unmonitored_loads_{year}.csv"
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](https://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2017.ipynb) for details.